In [ ]:
## Heuristic

In [1]:
import math
from typing import Dict, List

def cluster_aware_packing(
    activity_hours: Dict[str, float],
    capacities: Dict[str, int],
    clusters: List[List[str]],
    workweek: int = 38,
    oversub_limit: float = 0.1,
    eps: float = 1e-9
) -> List[Dict[str, int]]:
    """
    Same as before, maar de plaatsingsprioriteit is nu:
      1) same‑cluster zonder marge
      2) same‑cluster met marge
      3) any‑cluster zonder marge
      4) any‑cluster met marge
      5) nieuwe team
    """
    # 1. Compute required headcount per activity
    n = {a: math.ceil(hours / workweek) for a, hours in activity_hours.items()}
    # 2. Map each activity to its cluster index
    cluster_of = {a: cid for cid, grp in enumerate(clusters) for a in grp}

    teams: List[Dict[str, int]] = []
    rem_caps: List[float] = []  # remaining fractional capacity per team

    # 3. Repeat until all headcounts are zero
    while any(cnt > 0 for cnt in n.values()):
        # active activities
        active = {a: cnt for a, cnt in n.items() if cnt > 0}

        # a) full-team candidates
        full = [a for a, cnt in active.items() if cnt >= capacities[a]]
        if full:
            a = max(full, key=lambda a: capacities[a])
            chunk = capacities[a]
            chosen_cluster = cluster_of[a]
        else:
            # b) choose cluster by highest total fill-fraction
            f_cluster = []
            for cid, grp in enumerate(clusters):
                frac = sum(n[a] / capacities[a] for a in grp if n.get(a, 0) > 0)
                f_cluster.append((cid, frac))
            chosen_cluster = max(f_cluster, key=lambda x: x[1])[0]
            # c) pick activity within cluster by highest fraction
            candidates = {
                a: n[a] / capacities[a]
                for a in clusters[chosen_cluster] if n.get(a, 0) > 0
            }
            a = max(candidates, key=candidates.get)
            chunk = min(n[a], capacities[a])

        chunk_frac = chunk / capacities[a]
        placed = False

        # 1) same‑cluster, no oversubscription
        for idx, team in enumerate(teams):
            if any(team.get(b, 0) > 0 and cluster_of[b] == chosen_cluster for b in team):
                if rem_caps[idx] + eps >= chunk_frac and team.get(a, 0) + chunk <= capacities[a]:
                    team[a] = team.get(a, 0) + chunk
                    rem_caps[idx] -= chunk_frac
                    placed = True
                    break

        # 2) same‑cluster, with oversubscription
        if not placed:
            for idx, team in enumerate(teams):
                if any(team.get(b, 0) > 0 and cluster_of[b] == chosen_cluster for b in team):
                    if rem_caps[idx] + oversub_limit + eps >= chunk_frac and team.get(a, 0) + chunk <= capacities[a]:
                        team[a] = team.get(a, 0) + chunk
                        rem_caps[idx] -= chunk_frac
                        placed = True
                        break

        # 3) any team, no oversubscription
        if not placed:
            for idx, rem in enumerate(rem_caps):
                if rem + eps >= chunk_frac and teams[idx].get(a, 0) + chunk <= capacities[a]:
                    teams[idx][a] = teams[idx].get(a, 0) + chunk
                    rem_caps[idx] -= chunk_frac
                    placed = True
                    break

        # 4) any team, with oversubscription
        if not placed:
            for idx, rem in enumerate(rem_caps):
                if rem + oversub_limit + eps >= chunk_frac and teams[idx].get(a, 0) + chunk <= capacities[a]:
                    teams[idx][a] = teams[idx].get(a, 0) + chunk
                    rem_caps[idx] -= chunk_frac
                    placed = True
                    break

        # 5) else open new team
        if not placed:
            new_team = {act: 0 for act in capacities}
            new_team[a] = chunk
            teams.append(new_team)
            rem_caps.append(1.0 - chunk_frac)

        # e) update remaining headcount
        n[a] -= chunk

    return teams


if __name__ == '__main__':
    capacities = {
        'Receiving': 8, 'Putaway Manual': 12, 'Putaway Auto': 11,
        'Unloading Boxes': 14, 'Unloading Pallets': 11,
        'Loading': 10, 'Packing': 9, 'Picking Manual': 10,
        'Picking Auto': 12, 'Pallet Moves': 11, 'Box Moves': 11,
        'Cycle Count': 9, 'VAS': 10, 'Returns': 7
    }
    activity_hours = {
        'Receiving': 252.47,
        'Putaway Manual': 18.64,
        'Putaway Auto': 0,
        'Unloading Boxes': 0,
        'Unloading Pallets': 0,
        'Loading': 309.39,
        'Packing': 523.45,
        'Picking Manual': 483.54,
        'Picking Auto': 49.09,
        'Pallet Moves': 279.9,
        'Box Moves': 0,
        'Cycle Count': 8,
        'VAS': 169.83,
        'Returns': 0
    }
    clusters = [
        ['Receiving', 'Putaway Manual', 'Putaway Auto', 'Unloading Boxes', 'Unloading Pallets'],
        ['Loading', 'Packing', 'Picking Manual', 'Picking Auto'],
        ['Pallet Moves', 'Box Moves'],
        ['Cycle Count', 'VAS', 'Returns']
    ]

    teams = cluster_aware_packing(activity_hours, capacities, clusters)
    for i, team in enumerate(teams, 1):
        total = sum(team.values())
        fill_pct = 100 * sum(team[a] / capacities[a] for a in team)
        print(f"Team {i}: {total} people, approx fill ≈ {fill_pct:.1f}%")
        for act, num in team.items():
            if num:
                print(f"  {act}: {num}")


Team 1: 10 people, approx fill ≈ 100.0%
  Picking Manual: 10
Team 2: 9 people, approx fill ≈ 100.0%
  Packing: 9
Team 3: 11 people, approx fill ≈ 106.7%
  Loading: 9
  Picking Auto: 2
Team 4: 10 people, approx fill ≈ 105.6%
  Packing: 5
  VAS: 5
Team 5: 9 people, approx fill ≈ 106.9%
  Receiving: 7
  Putaway Manual: 1
  Cycle Count: 1
Team 6: 11 people, approx fill ≈ 102.7%
  Picking Manual: 3
  Pallet Moves: 8


In [ ]:
## Interaction matrix

In [2]:
activities = [
    'Receiving', 'Putaway Manual', 'Putaway Auto', 'Unloading Boxes',
    'Unloading Pallets', 'Loading', 'Packing', 'Picking Manual',
    'Picking Auto', 'Pallet Moves', 'Box Moves', 'Cycle Count',
    'VAS', 'Returns'
]

# 🟩 Vul hier je interactiescores in (tussen 0 en 10)
raw_interaction = {
    ('Receiving', 'Putaway Manual'): 6,
    ('Receiving', 'Putaway Auto'): 6,
    ('Receiving', 'Unloading Boxes'): 8,
    ('Receiving', 'Unloading Pallets'): 8,
    ('Receiving', 'Loading'): 5,
    ('Receiving', 'Packing'): 3,
    ('Receiving', 'Picking Manual'): 2,
    ('Receiving', 'Picking Auto'): 2,
    ('Receiving', 'Pallet Moves'): 4,
    ('Receiving', 'Box Moves'): 4,
    ('Receiving', 'Cycle Count'): 4,
    ('Receiving', 'VAS'): 4,
    ('Receiving', 'Returns'): 5,

    ('Putaway Manual', 'Putaway Auto'): 10,
    ('Putaway Manual', 'Unloading Boxes'): 7,
    ('Putaway Manual', 'Unloading Pallets'): 7,
    ('Putaway Manual', 'Loading'): 5,
    ('Putaway Manual', 'Packing'): 5,
    ('Putaway Manual', 'Picking Manual'): 7,
    ('Putaway Manual', 'Picking Auto'): 7,
    ('Putaway Manual', 'Pallet Moves'): 6,
    ('Putaway Manual', 'Box Moves'): 6,
    ('Putaway Manual', 'Cycle Count'): 3,
    ('Putaway Manual', 'VAS'): 3,
    ('Putaway Manual', 'Returns'): 2,

    ('Putaway Auto', 'Unloading Boxes'): 7,
    ('Putaway Auto', 'Unloading Pallets'): 7,
    ('Putaway Auto', 'Loading'): 5,
    ('Putaway Auto', 'Packing'): 5,
    ('Putaway Auto', 'Picking Manual'): 7,
    ('Putaway Auto', 'Picking Auto'): 7,
    ('Putaway Auto', 'Pallet Moves'): 6,
    ('Putaway Auto', 'Box Moves'): 6,
    ('Putaway Auto', 'Cycle Count'): 3,
    ('Putaway Auto', 'VAS'): 3,
    ('Putaway Auto', 'Returns'): 2,

    ('Unloading Boxes', 'Unloading Pallets'): 10,
    ('Unloading Boxes', 'Loading'): 8,
    ('Unloading Boxes', 'Packing'): 5,
    ('Unloading Boxes', 'Picking Manual'): 4,
    ('Unloading Boxes', 'Picking Auto'): 4,
    ('Unloading Boxes', 'Pallet Moves'): 3,
    ('Unloading Boxes', 'Box Moves'): 3,
    ('Unloading Boxes', 'Cycle Count'): 3,
    ('Unloading Boxes', 'VAS'): 3,
    ('Unloading Boxes', 'Returns'): 2,

    ('Unloading Pallets', 'Loading'): 8,
    ('Unloading Pallets', 'Packing'): 5,
    ('Unloading Pallets', 'Picking Manual'): 4,
    ('Unloading Pallets', 'Picking Auto'): 4,
    ('Unloading Pallets', 'Pallet Moves'): 3,
    ('Unloading Pallets', 'Box Moves'): 3,
    ('Unloading Pallets', 'Cycle Count'): 3,
    ('Unloading Pallets', 'VAS'): 3,
    ('Unloading Pallets', 'Returns'): 2,

    ('Loading', 'Packing'): 8,
    ('Loading', 'Picking Manual'): 5,
    ('Loading', 'Picking Auto'): 5,
    ('Loading', 'Pallet Moves'): 4,
    ('Loading', 'Box Moves'): 4,
    ('Loading', 'Cycle Count'): 6,
    ('Loading', 'VAS'): 3,
    ('Loading', 'Returns'): 5,

    ('Packing', 'Picking Manual'): 6,
    ('Packing', 'Picking Auto'): 6,
    ('Packing', 'Pallet Moves'): 2,
    ('Packing', 'Box Moves'): 2,
    ('Packing', 'Cycle Count'): 5,
    ('Packing', 'VAS'): 5,
    ('Packing', 'Returns'): 5,

    ('Picking Manual', 'Picking Auto'): 10,
    ('Picking Manual', 'Pallet Moves'): 7,
    ('Picking Manual', 'Box Moves'): 7,
    ('Picking Manual', 'Cycle Count'): 3,
    ('Picking Manual', 'VAS'): 3,
    ('Picking Manual', 'Returns'): 2,

    ('Picking Auto', 'Pallet Moves'): 7,
    ('Picking Auto', 'Box Moves'): 7,
    ('Picking Auto', 'Cycle Count'): 3,
    ('Picking Auto', 'VAS'): 3,
    ('Picking Auto', 'Returns'): 2,

    ('Pallet Moves', 'Box Moves'): 10,
    ('Pallet Moves', 'Cycle Count'): 4,
    ('Pallet Moves', 'VAS'): 1,
    ('Pallet Moves', 'Returns'): 4,

    ('Box Moves', 'Cycle Count'): 4,
    ('Box Moves', 'VAS'): 1,
    ('Box Moves', 'Returns'): 4,

    ('Cycle Count', 'VAS'): 6,
    ('Cycle Count', 'Returns'): 7,

    ('VAS', 'Returns'): 2
}

# 🔁 Automatisch symmetrisch maken + diagonaal op 0
interaction = {}
for (i, j), val in raw_interaction.items():
    interaction[i, j] = val
    interaction[j, i] = val
for a in activities:
    interaction[(a, a)] = 0


In [ ]:
## MIP model

In [3]:
import math
import random
import gurobipy as gp
from gurobipy import GRB

def solve_mip_with_interaction(activity_hours, capacities, clusters,
                               interaction,  # 🔹 extra argument
                               delta=0.1, time_limit=300, alpha=0.55):

    """
    MIP met extra term om activiteiten met hoge interactie samen in teams te groeperen.
    """

    W = 38.0
    n = {a: math.ceil(activity_hours.get(a, 0) / W) for a in capacities}
    activities = list(capacities.keys())
    Tmax = sum(n.values())

    m = gp.Model("team_assignment_with_interaction")
    m.Params.TimeLimit = time_limit
    m.Params.MIPGap = 0.01
    m.Params.OutputFlag = 0  # stil uitvoeren

    # Variabelen
    y = m.addVars(Tmax, vtype=GRB.BINARY, name="y")
    x = m.addVars(capacities.keys(), range(Tmax), vtype=GRB.INTEGER, name="x")
    z = m.addVars(capacities.keys(), range(Tmax), vtype=GRB.BINARY, name="z")
    u = m.addVars(len(clusters), range(Tmax), vtype=GRB.BINARY, name="u")

    # 🔸 Nieuwe variabele: w[i,j,t] = 1 als i en j samen in team t zitten
    w = m.addVars(activities, activities, range(Tmax), vtype=GRB.BINARY, name="w")

    # Constraints
    for a in capacities:
        m.addConstr(x.sum(a, '*') == n[a], name=f"demand_{a}")

    for t in range(Tmax):
        m.addConstr(
            gp.quicksum(x[a, t] / capacities[a] for a in capacities)
            <= (1 + delta) * y[t],
            name=f"cap_team_{t}"
        )
        for a in capacities:
            m.addConstr(x[a, t] <= capacities[a] * y[t], name=f"act_cap_{a}_{t}")
            m.addConstr(x[a, t] <= n[a] * z[a, t], name=f"x_z_up_{a}_{t}")
            m.addConstr(z[a, t] <= y[t], name=f"z_y_{a}_{t}")

    for i, Ci in enumerate(clusters):
        for t in range(Tmax):
            for a in Ci:
                m.addConstr(z[a, t] <= u[i, t], name=f"z_u_{i}_{a}_{t}")
            m.addConstr(gp.quicksum(z[a, t] for a in Ci) >= u[i, t],
                        name=f"u_z_{i}_{t}")

    for a in capacities:
        m.addConstr(
            gp.quicksum(z[a, t] for t in range(Tmax)) <= n[a],
            name=f"max_teams_{a}"
        )

    # 🔸 Interactie koppelen aan z-variabelen
    for t in range(Tmax):
        for i in activities:
            for j in activities:
                if i != j:
                    m.addConstr(w[i, j, t] <= z[i, t], name=f"w_z1_{i}_{j}_{t}")
                    m.addConstr(w[i, j, t] <= z[j, t], name=f"w_z2_{i}_{j}_{t}")
                    m.addConstr(w[i, j, t] >= z[i, t] + z[j, t] - 1, name=f"w_z3_{i}_{j}_{t}")

    # Doelfuncties
    m.setObjectiveN(y.sum(), index=0, priority=3, name="min_teams")
    m.setObjectiveN(u.sum(), index=1, priority=0, name="min_clusters")
    m.setObjectiveN(z.sum(), index=2, priority=2, name="min_mixes")

    # 🔸 Toegevoegde interactiedoelstelling (negatief = maximeren)
    interaction_score = gp.quicksum(
        interaction[i, j] * w[i, j, t]
        for i in activities for j in activities if i != j
        for t in range(Tmax)
    )
    m.setObjectiveN(-alpha * interaction_score, index=3, priority=1, name="max_interactions")

    # Optimaliseren
    m.optimize()

    used_teams = 0
    team_composition = {}
    
    print("")  # lege regel
    for t in range(Tmax):
        if y[t].X > 0.5:
            used_teams += 1
            team_composition[t] = {}
            fill_fraction = 0.0
            total_people = 0
    
            for a in activities:
                assigned = int(round(x[a, t].X))
                if assigned > 0:
                    team_composition[t][a] = assigned
                    fill_fraction += assigned / capacities[a]
                    total_people += assigned
    
            print(f"\n🧩 Team {t + 1}: {total_people} mensen, approx fill ≈ {100 * fill_fraction:.1f}%")
            for a, num in team_composition[t].items():
                print(f"  - {a}: {num} FTE")
    
    print(f"\nAantal gebruikte teams: {used_teams}")
    return used_teams, m




# Test-aanroep
if __name__ == '__main__':
    capacities = {
        'Receiving': 8, 'Putaway Manual': 12, 'Putaway Auto': 11,
        'Unloading Boxes': 14, 'Unloading Pallets': 11,
        'Loading': 10, 'Packing': 9, 'Picking Manual': 10,
        'Picking Auto': 12, 'Pallet Moves': 11, 'Box Moves': 11,
        'Cycle Count': 9, 'VAS': 10, 'Returns': 7
    }
    
    activity_hours = {
            'Receiving': 252.47,
            'Putaway Manual': 18.64,
            'Putaway Auto': 0,
            'Unloading Boxes': 0,
            'Unloading Pallets': 0,
            'Loading': 309.39,
            'Packing': 523.45,
            'Picking Manual': 483.54,
            'Picking Auto': 49.09,
            'Pallet Moves': 279.9,
            'Box Moves': 0,
            'Cycle Count': 8,
            'VAS': 169.83,
            'Returns': 0
    }

    clusters = [
        ['Receiving', 'Putaway Manual', 'Putaway Auto', 'Unloading Boxes', 'Unloading Pallets'],
        ['Loading', 'Packing', 'Picking Manual', 'Picking Auto'],
        ['Pallet Moves', 'Box Moves'],
        ['Cycle Count', 'VAS', 'Returns']
    ]

    used, model = solve_mip_with_interaction(
    activity_hours, capacities, clusters, interaction
)
    print(f"Minimaal aantal teams (met interacties): {used}")


Set parameter Username
Set parameter LicenseID to value 2653856
Academic license - for non-commercial use only - expires 2026-04-17
Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0.01


🧩 Team 2: 11 mensen, approx fill ≈ 102.7%
  - Picking Manual: 3 FTE
  - Pallet Moves: 8 FTE

🧩 Team 3: 9 mensen, approx fill ≈ 100.0%
  - Packing: 9 FTE

🧩 Team 22: 10 mensen, approx fill ≈ 105.6%
  - Packing: 5 FTE
  - VAS: 5 FTE

🧩 Team 24: 11 mensen, approx fill ≈ 106.7%
  - Loading: 9 FTE
  - Picking Auto: 2 FTE

🧩 Team 45: 9 mensen, approx fill ≈ 106.9%
  - Receiving: 7 FTE
  - Putaway Manual: 1 FTE
  - Cycle Count: 1 FTE

🧩 Team 60: 10 mensen, approx fill ≈ 100.0%
  - Picking Manual: 10 FTE

Aantal gebruikte teams: 6
Minimaal aantal teams (met interacties): 6


In [ ]:
## Benchmark script

In [ ]:
import time
import random
import matplotlib.pyplot as plt

def filter_interaction_matrix(full_matrix, acts):
    """Filter de volledige interactiematrix tot enkel paren in 'acts'."""
    filtered = {}
    for i in acts:
        for j in acts:
            filtered[i, j] = full_matrix.get((i, j), 0)
    return filtered

full_interaction_matrix = {}
for (i, j), val in raw_interaction.items():
    full_interaction_matrix[i, j] = val
    full_interaction_matrix[j, i] = val
for a in activities:
    full_interaction_matrix[(a, a)] = 0



def benchmark(n_instances=30, min_act=5, max_act=14, mip_time_limit=10):
    results = []
    for idx in range(1, n_instances+1):
        # 1) Kies een random aantal activiteiten
        n_act = random.randint(min_act, max_act)
        acts  = random.sample(list(capacities), n_act)

        # 2) Kies een gemiddelde urenwaarde met bias naar lage kant
        u = random.random()
        avg_hours = 50 + (300-50) * (u**2)    # meer gewicht bij lagere gemiddelden
        sigma     = avg_hours * 0.3           # 30% standaarddeviatie

        # 3) Ken per activiteit uren toe rond avg_hours
        hours = {}
        for a in acts:
            h = int(random.gauss(avg_hours, sigma))
            # clamp naar [50,2000]
            h = max(50, min(4000, h))
            hours[a] = h

        # 4) Filter vaste clusters op gekozen activiteiten
        clusters_inst = [[a for a in cl if a in acts] for cl in clusters]

        # Optioneel: toon de instantie
        print(f"\nInstance {idx}: {n_act} activiteiten, avg≈{avg_hours:.0f}")
        for a in acts:
            print(f"  - {a}: {hours[a]} uur")
        print("  Clusters:", [cl for cl in clusters_inst if cl])

        # 5) Run heuristiek
        t0 = time.perf_counter()
        teams_h = cluster_aware_packing(hours, {a: capacities[a] for a in acts}, clusters_inst)
        t_h = time.perf_counter() - t0
        H = len(teams_h)

        # 6) Run MIP
        t0 = time.perf_counter()
        interaction = filter_interaction_matrix(full_interaction_matrix, acts)
        t0 = time.perf_counter()
        OPT, _ = solve_mip_with_interaction(
            hours,
            {a: capacities[a] for a in acts},
            clusters_inst,
            interaction=interaction,
            delta=0.1,
            time_limit=mip_time_limit,
            alpha=0.5  # of ander gewicht naar wens
        )

        t_m = time.perf_counter() - t0

        results.append((H, OPT, t_h, t_m))

    return results



if __name__ == "__main__":
    # Run benchmark
    res = benchmark(n_instances=50, min_act=5, max_act=14, mip_time_limit=30)

    # Performance profile
    ratios = [H/OPT for H,OPT,_,_ in res]
    taus   = sorted(set(ratios))
    perf   = [sum(1 for r in ratios if r <= tau)/len(ratios) for tau in taus]

    plt.plot(taus, perf, marker='o')
    plt.xlabel('Performance ratio τ = Heuristic / OPT')
    plt.ylabel('Fraction of instances ≤ τ')
    plt.title('Performance Profile')
    plt.grid(True)
    plt.show()

    # Samenvatting gaps en tijden
    gaps    = [H-OPT for H,OPT,_,_ in res]
    times_h = [t for *_,t,_ in res]
    times_m = [t for *_,_,t in res]
    print(f"Avg gap        : {sum(gaps)/len(gaps):.2f} teams")
    print(f"Max gap        : {max(gaps)} teams")
    print(f"Avg time (heu) : {sum(times_h)/len(times_h):.2f}s")
    print(f"Avg time (MIP) : {sum(times_m)/len(times_m):.2f}s")



Instance 1: 6 activiteiten, avg≈53
  - Unloading Pallets: 69 uur
  - Returns: 52 uur
  - Pallet Moves: 79 uur
  - Loading: 52 uur
  - Box Moves: 63 uur
  - Packing: 86 uur
  Clusters: [['Unloading Pallets'], ['Loading', 'Packing'], ['Pallet Moves', 'Box Moves'], ['Returns']]
Set parameter TimeLimit to value 30
Set parameter MIPGap to value 0.01


🧩 Team 9: 5 mensen, approx fill ≈ 45.5%
  - Pallet Moves: 3 FTE
  - Box Moves: 2 FTE

🧩 Team 14: 9 mensen, approx fill ≈ 100.1%
  - Unloading Pallets: 2 FTE
  - Returns: 2 FTE
  - Loading: 2 FTE
  - Packing: 3 FTE

Aantal gebruikte teams: 2

Instance 2: 10 activiteiten, avg≈172
  - Picking Auto: 165 uur
  - Packing: 210 uur
  - Pallet Moves: 145 uur
  - Receiving: 200 uur
  - Putaway Manual: 139 uur
  - Unloading Pallets: 97 uur
  - Unloading Boxes: 119 uur
  - Cycle Count: 171 uur
  - Box Moves: 134 uur
  - VAS: 104 uur
  Clusters: [['Receiving', 'Putaway Manual', 'Unloading Boxes', 'Unloading Pallets'], ['Packing', 'Picking Auto'], ['Pallet